In [31]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


### Pre-requisties

In [47]:
a = tf.truncated_normal([16,128,128,3])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.shape(a))

array([ 16, 128, 128,   3], dtype=int32)

In [48]:
b = tf.reshape(a,[16,49152])
sess.run(tf.shape(b))

array([   16, 49152], dtype=int32)